In [1]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1646707271339_0010,pyspark,idle,Link,Link,


In [2]:
sc.install_pypi_package("pip==22.0.4")
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("Pillow")
sc.install_pypi_package("pyarrow")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1646707271339_0011,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/4d/16/0a14ca596f30316efd412a60bdfac02a7259bf8673d4d917dc60b9a21812/pip-22.0.4-py3-none-any.whl
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1

  Using cached pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl (10.4 MB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)

  Using cached Pillow-9.0.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)

  Using cached pyarrow-7.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.5
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /tmp/1646728247377-0
    Can't uninstall 'numpy'. No files were found to uninstall.

  Cache entry deserialization failed, entry ignored



ER

In [3]:
# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import tensorflow as tf
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql import functions as F

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import warnings
warnings.filterwarnings('ignore')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# This is just to take a look at the dataset, this dataframe is not really needed
# since the training data will be obtained from the images themselves

# df_masks = spark.read.csv('s3://pguevara-bdcc2022/mask_project/medical-masks-part1/df_masks.csv', header=True)
# df_masks.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
df_masks = spark.read.csv('s3://medical-masks-bdcc/df.csv', header=True)
df_masks = df_masks.filter(df_masks.size_mb < 1.5).toPandas()
df_masks['size_mb'] = df_masks['size_mb'].astype(float)
df_masks['new_path'] = "s3://medical-masks-bdcc/" + df_masks['name'].astype(str)
file_list = df_masks['new_path'].tolist()

print(f'Total size of dataset: {df_masks["size_mb"].sum():.4f} GB')
print(f'Total number of images: {len(df_masks)}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total size of dataset: 15521.2794 GB
Total number of images: 16394

In [61]:
# Modify the wildcards in the below filepath to control which/how many images are loaded
# into the images dataframe

url = 's3://medical-masks-bdcc/??????_?_??????*.jpg'

images = (spark.read.format("binaryFile").
          option("pathGlobFilter", "*.jpg").
          load(url))
images = (images.filter(F.col('path').isin(file_list)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
images.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

In [63]:
images.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16394

In [64]:
# images.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
model = EfficientNetB0(include_top=False)
# model = ResNet50(include_top=False)
model.summary()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, None, None, 3 7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, None, None, 3 0           normalization_1[0][0]            
_____________________________________________________________________________________

In [66]:
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
  """
  Returns a ResNet50/EfficientNetB0 model with top layer removed and broadcasted pretrained weights.
  """
  model = EfficientNetB0(weights=None, include_top=False)
#   model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
def preprocess(content):
  """
  Preprocesses raw image bytes for prediction.
  """
#   try:
  img = Image.open(io.BytesIO(content)).resize([128, 128])
  arr = img_to_array(img)
  return preprocess_input(arr)
#   except:
#       return np.zeros([128, 128])


def featurize_series(model, content_series):
  """
  Featurize a pd.Series of raw images using the input model.
  :return: a pd.Series of image features
  """
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  # For some layers, output features will be multi-dimensional tensors.
  # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
  output = [p.flatten() for p in preds]
  return pd.Series(output)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  '''
  This method is a Scalar Iterator pandas UDF wrapping our featurization function.
  The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).
  
  :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
  '''
  # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
  # for multiple data batches.  This amortizes the overhead of loading big models.
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
# Pandas UDFs on large records (e.g., very large images) can run into Out Of Memory (OOM) errors.
# If you hit such errors in the cell below, try reducing the Arrow batch size via `maxRecordsPerBatch`.
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "512")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
images.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1910

In [71]:
# We can now run featurization on our entire Spark DataFrame.
# NOTE: This can take a long time (about 10 minutes) since it applies a large model to the full dataset.
features_df = (images.repartition(images.rdd.getNumPartitions()*2).
               select(col("path"), featurize_udf("content").
                      alias("features")))
# features_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
features_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)

In [73]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.sql import functions as F

list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

df = (features_df.
      withColumn('label', F.substring('path', 32, 1).cast('int')).
      withColumn('features', list_to_vector_udf(F.col('features'))))
# df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
df.select('label').groupby('label').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
|label|count|
+-----+-----+
|    1| 3975|
|    3| 4144|
|    4| 4212|
|    2| 4063|
+-----+-----+

In [75]:
# This is just to save a copy of the featurized dataset (before train-test splitting)
# in parquet format on the s3 bucket
# df.write.mode("overwrite").parquet('s3://pguevara-bdcc2022/mask_project/medical-masks-part1/tmp/dataset')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)

In [77]:
df_train, df_test = df.randomSplit([0.75, 0.25])
# print(df_test.rdd.getNumPartitions())
# print(df_train.rdd.getNumPartitions())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
from pyspark.ml.classification import RandomForestClassifier

rf_class = RandomForestClassifier(featuresCol='features',
                                  labelCol='label',
                                  maxBins=100, seed=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
rf_trained = rf_class.fit(df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:

  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1646707271339_0011/container_1646707271339_0011_01_000028/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/mnt1/yarn/usercache/livy/appcache/application_1646707271339_0011/container_1646707271339_0011_01_000028/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/mnt1/yarn/usercache/livy/appcache/application_1646707271339_0011/container_1646707271339_0011_01_000028/pyspark.zip/pyspark/sql/pandas/serializers.py", line 273, in dump_stream
    return ArrowStreamSerializer.dump_stream(self, init_stream_yield_batches(), stream)
  File "/mnt1/yarn/usercache/livy/appcache/application_1646707271339_0011/container_1646707271339_0011_01_000028/pyspark.zip/pyspark/sql/pandas/serializers.py", line 81, in dump_stream
    fo

In [80]:
df_predictions = rf_trained.transform(df_test)
# df_predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'rf_trained' is not defined
Traceback (most recent call last):
NameError: name 'rf_trained' is not defined



In [81]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(df_predictions.select("prediction", "label"))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_predictions' is not defined
Traceback (most recent call last):
NameError: name 'df_predictions' is not defined



In [82]:
# df_predictions.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
# df_predictions.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
# df_predictions.filter(F.col('label')==F.col('prediction')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
rf_trained.save('EffNet_RF_mask_trained')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'rf_trained' is not defined
Traceback (most recent call last):
NameError: name 'rf_trained' is not defined

